Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

Preparing the dataset

In [2]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

trainset = torchvision.datasets.CIFAR10(root = './data',train = True,download = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset,batch_size = 4,shuffle = True, num_workers = 2)

testset = torchvision.datasets.CIFAR10(root = './data',train = False,download = True,transform = transform)
testloader = torch.utils.data.DataLoader(testset,batch_size = 4, shuffle = False, num_workers = 2)

classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')


100%|██████████| 170M/170M [00:05<00:00, 30.0MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


CNN Architecture

In [8]:
class Network(nn.Module) :
  def __init__(self):
    super(Network,self).__init__()

    self.Conv1 = nn.Conv2d(3,32,kernel_size = 3,padding = 1)
    self.bn1 = nn.BatchNorm2d(32)
    self.Conv2 = nn.Conv2d(32,64,kernel_size = 3,padding = 1)
    self.bn2 = nn.BatchNorm2d(64)
    self.Conv3 = nn.Conv2d(64,128,kernel_size = 3, padding = 1)
    self.bn3 = nn.BatchNorm2d(128)

    self.pool = nn.MaxPool2d(2,2)
    self.gap = nn.AdaptiveAvgPool2d((1,1))

    self.fc = nn.Linear(128,10)
    self.dropout = nn.Dropout(0.5)

  def forward(self,x):
      x = self.pool(F.relu(self.bn1(self.Conv1(x))))
      x = self.pool(F.relu(self.bn2(self.Conv2(x))))
      x = self.pool(F.relu(self.bn3(self.Conv3(x))))
      x = self.gap(x)
      x = torch.flatten(x,1)
      x = self.dropout(x)
      x = self.fc(x)
      return x

net = Network()

Loss Function and Optimizer

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=0.0001)

Training the Network

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)
for epoch in range(5):
  net.train()
  running_loss = 0.0

  for i,data in enumerate(trainloader,0):
    inputs,labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    outputs = net(inputs)
    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    if i % 2000 == 1999:
      print(f"[Epoch {epoch + 1}, Batch {i + 1}] Loss: {running_loss / 2000:.3f}")
      running_loss = 0.0
print("Training Finished")

[Epoch 1, Batch 2000] Loss: 1.544
[Epoch 1, Batch 4000] Loss: 1.537
[Epoch 1, Batch 6000] Loss: 1.521
[Epoch 1, Batch 8000] Loss: 1.528
[Epoch 1, Batch 10000] Loss: 1.501
[Epoch 1, Batch 12000] Loss: 1.500
[Epoch 2, Batch 2000] Loss: 1.469
[Epoch 2, Batch 4000] Loss: 1.469
[Epoch 2, Batch 6000] Loss: 1.453
[Epoch 2, Batch 8000] Loss: 1.483
[Epoch 2, Batch 10000] Loss: 1.476
[Epoch 2, Batch 12000] Loss: 1.431
[Epoch 3, Batch 2000] Loss: 1.436
[Epoch 3, Batch 4000] Loss: 1.429
[Epoch 3, Batch 6000] Loss: 1.424
[Epoch 3, Batch 8000] Loss: 1.415
[Epoch 3, Batch 10000] Loss: 1.432
[Epoch 3, Batch 12000] Loss: 1.419
[Epoch 4, Batch 2000] Loss: 1.381
[Epoch 4, Batch 4000] Loss: 1.383
[Epoch 4, Batch 6000] Loss: 1.386
[Epoch 4, Batch 8000] Loss: 1.404
[Epoch 4, Batch 10000] Loss: 1.387
[Epoch 4, Batch 12000] Loss: 1.363
[Epoch 5, Batch 2000] Loss: 1.367
[Epoch 5, Batch 4000] Loss: 1.373
[Epoch 5, Batch 6000] Loss: 1.358
[Epoch 5, Batch 8000] Loss: 1.354
[Epoch 5, Batch 10000] Loss: 1.354
[Epoc

Testing the Network

In [17]:
correct = 0
total = 0

net.eval()
with torch.no_grad():
  for data in testloader:
    inputs,labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data,1)

    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f} %')

Accuracy of the network on the 10000 test images: 61.80 %


Optical Extension

In [19]:
from torchvision import models

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

net = models.resnet18(pretrained = True)
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 10)

net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=0.0001)

for epoch in range(5):
  net.train()
  running_loss = 0.0

  for i,data in enumerate(trainloader,0):
    inputs,labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    outputs = net(inputs)
    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    if i % 2000 == 1999:
      print(f"[Epoch {epoch + 1}, Batch {i + 1}] Loss: {running_loss / 2000:.3f}")
      running_loss = 0.0
print("Training Finished")

correct = 0
total = 0

net.eval()
with torch.no_grad():
  for data in testloader:
    inputs,labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data,1)

    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f} %')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 191MB/s]


[Epoch 1, Batch 2000] Loss: 1.858
[Epoch 1, Batch 4000] Loss: 1.692
[Epoch 1, Batch 6000] Loss: 1.601
[Epoch 1, Batch 8000] Loss: 1.509
[Epoch 1, Batch 10000] Loss: 1.430
[Epoch 1, Batch 12000] Loss: 1.363
[Epoch 2, Batch 2000] Loss: 1.282
[Epoch 2, Batch 4000] Loss: 1.240
[Epoch 2, Batch 6000] Loss: 1.224
[Epoch 2, Batch 8000] Loss: 1.168
[Epoch 2, Batch 10000] Loss: 1.143
[Epoch 2, Batch 12000] Loss: 1.108
[Epoch 3, Batch 2000] Loss: 1.028
[Epoch 3, Batch 4000] Loss: 1.019
[Epoch 3, Batch 6000] Loss: 0.979
[Epoch 3, Batch 8000] Loss: 0.995
[Epoch 3, Batch 10000] Loss: 0.944
[Epoch 3, Batch 12000] Loss: 0.948
[Epoch 4, Batch 2000] Loss: 0.856
[Epoch 4, Batch 4000] Loss: 0.880
[Epoch 4, Batch 6000] Loss: 0.859
[Epoch 4, Batch 8000] Loss: 0.845
[Epoch 4, Batch 10000] Loss: 0.844
[Epoch 4, Batch 12000] Loss: 0.840
[Epoch 5, Batch 2000] Loss: 0.778
[Epoch 5, Batch 4000] Loss: 0.757
[Epoch 5, Batch 6000] Loss: 0.750
[Epoch 5, Batch 8000] Loss: 0.734
[Epoch 5, Batch 10000] Loss: 0.772
[Epoc